## 1. Setup StockX

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()


ib = IB()
port = 7496
port = 4002
ib.connect('127.0.0.1', port, clientId=15)
# sx  = stock.StockX(ib, 'TSLA')
# spy = stock.StockX(ib, 'SPY')

<IB connected to 127.0.0.1:4002 clientId=15>

# -------------------------------------

In [2]:
import stock
from strategies.preset_strats import TAPresets1D, TAPresets1H


# symbols to check for and why they are chosen. Gaps not on this days but charts have good previous gaps.
# symbol = 'SATX', date = '2025-04-01', size = '1 day', reason = 'Gapps'
# symbol = 'ECX',  date = '2025-04-01', size = '1 day', reason = 'Big Gap down and gap up '
# symbol = 'ANIX', date = '2025-04-01', size = '1 day', reason = 'Gap over resistance'
# symbol = 'SLQT', date = '2025-04-01', size = '1 day', reason = 'Big gap up '
# symbol = 'PRGS', date = '2025-04-01', size = '1 day', reason = 'Big Gap down and gap up '
# symbol = 'CFFN', date = '2025-04-01', size = '1 day', reason = 'Big Gap down and gap up '
# symbol = 'SRAD', date = '2025-04-01', size = '1 day', reason = 'Big gap up '

# TODO - Fix gaps. GPivs and Gapped past Pivots
# TODO - Fix RS needs to be Score 0,100 .  add score to TA maybe 
# TODO - Not sure Gapped Bars is working correctly. GapWRBs

sx  = stock.StockX(ib, 'UVIX') 
spy = stock.StockX(ib, 'SPY')


def RUN_DAILY(self, ls:str='', spy:object=None, isMarket:bool=False, displayCharts:bool=False, printStats:bool=False, forceDownload:bool=False):
    print(f"------------ StockX::RUN_DAILY: {self.symbol} {self.ls}------------------------------------------------------------------------")

    self.set_ls(ls)
    if isMarket:
        self.setup_frame('1 day', 'ohlcv', duration="200 D", endDateTime='now', isDayFrame=True,      taPresets=TAPresets1D(ls=self.ls, isSpy=True, lookBack=150))
        self.setup_frame('1 hour', 'ohlcv', duration="15 D", endDateTime='now', isIntradayFrame=True, taPresets=TAPresets1H(ls=self.ls, isSpy=True, lookBack=150))
        self.frames['1 day'].run_ta()
        self.frames['1 hour'].run_ta()

    else:
        self.setup_frame('1 day', 'ohlcv', duration="200 D", endDateTime='now', isDayFrame=True,      taPresets=TAPresets1D(ls=self.ls, lookBack=100))
        self.setup_frame('1 hour', 'ohlcv', duration="15 D", endDateTime='now', isIntradayFrame=True, taPresets=TAPresets1H(ls=self.ls, lookBack=100, volChgPctThreshold=50))
        self.import_all_market_data(spy)
        self.frames['1 day'].run_ta()
        self.frames['1 hour'].run_ta()

 

ls= 'LONG'
RUN_DAILY(spy, ls=ls, isMarket=True)
RUN_DAILY(sx, ls=ls, spy=spy, isMarket=False)

------------ StockX::RUN_DAILY: SPY ------------------------------------------------------------------------
1D :: LONG :: 150
SPY 1 day 200 D :: 2024-06-21 00:00:00 to 2025-04-07 00:00:00 rows: 199, missing: 0, Data stored as 1 day and resampled to 1 day
SPY 1 hour 15 D :: 2025-03-19 00:00:00 to 2025-04-07 11:00:00 rows: 220, missing: 0, Data stored as 1 hour and resampled to 1 hour
------------ StockX::RUN_DAILY: UVIX ------------------------------------------------------------------------
1D :: LONG :: 100
UVIX 1 day 200 D :: 2024-06-21 00:00:00 to 2025-04-07 00:00:00 rows: 199, missing: 0, Data stored as 1 day and resampled to 1 day
UVIX 1 hour 15 D :: 2025-03-19 00:00:00 to 2025-04-07 11:00:00 rows: 220, missing: 1, Data stored as 1 hour and resampled to 1 hour


In [3]:
# spy.get_frame_data('1 day').columns #  .taPresets.BarSW
# sx.get_frame_data('1 day').columns #  .taPresets.BarSW
spy.get_frame('1 day').taPresets.__dict__['ta_list'] # .RoomToMove

[ATR(column=None, name='ATR_14', names=['ATR_14'], chartArgs=ChartArgItem(style={}, chartType='', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=14),
 HPLP(column=None, name=None, names=['HP_hi_10', 'LP_lo_10'], chartArgs=ChartArgItem(style=[{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chartType='points', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=10),
 HPLP(column=None, name=None, names=['HP_hi_3', 'LP_lo_3'], chartArgs=ChartArgItem(style=[{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chartType='points', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=3),
 SupResAllRows(column=None, name=None, names=['Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper', 'Sup_2_Lower'], chartArgs=ChartArgItem(style=[{'dash': 'solid', 'main_line_colo

In [4]:
sx.frames['1 day'].taPresets.get_ta_list()
sx.frames['1 hour'].taPresets.get_ta_list()


[ATR(column=None, name='ATR_14', names=['ATR_14'], chartArgs=ChartArgItem(style={}, chartType='', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=14),
 HPLP(column=None, name=None, names=['HP_hi_10', 'LP_lo_10'], chartArgs=ChartArgItem(style=[{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chartType='points', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=10),
 HPLP(column=None, name=None, names=['HP_hi_3', 'LP_lo_3'], chartArgs=ChartArgItem(style=[{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chartType='points', row=1, nameCol=None, columns=None, mask=None, name=None), hi_col='high', lo_col='low', span=3),
 SupResAllRows(column=None, name=None, names=['Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper', 'Sup_2_Lower'], chartArgs=ChartArgItem(style=[{'dash': 'solid', 'main_line_colo

In [5]:

s_1D_sigs = spy.get_frame('1 day').taPresets.s_1D.sigs
s_1D_sigs_names = [s_1D_sigs[i].name for i in range(len(s_1D_sigs))]
s_1D_sigs_names

s_1D_vals = spy.get_frame_data('1 day')[s_1D_sigs_names]
s_1D_vals.iloc[-1]

RTM_L_Res_1_Lower    1.865278
L_GapWRBs                 0.0
L_GPivs                   0.0
L_GPP                     0.0
Name: 2025-04-08 00:00:00, dtype: object

In [6]:
spy.get_frame_data('1 day').tail(20).iloc[:-9]

,open,high,low,close,volume,average,barCount,ATR_14,HP_hi_10,LP_lo_10,...,L_GPivs,L_GPP,VAL_L_L_GapWRBs_>_50,VAL_L_L_GPivs_>_1,VAL_L_L_GPP_>_80,VAL_L_RTM_L_Res_1_Lower_>_1,S_s_1D_mean,S_s_1D_mean_>=_50,S_sv_1D_mean,S_sv_1D_mean_>=_100
date,,,,,,,,,,,,,,,,,,,,,
2025-03-12,562.17,563.11,553.69,558.87,418632.0,559.312,229036.0,11.332857,NaN,NaN,...,0.0,0.0,0,0,0,100,2.35,0.0,100,100
2025-03-13,558.49,559.11,549.68,551.42,400069.0,553.292,222477.0,11.232857,NaN,549.68,...,0.0,0.0,0,0,0,100,5.69,0.0,100,100
2025-03-14,556.11,563.83,555.50,562.81,364382.0,560.546,191800.0,11.360714,NaN,NaN,...,0.0,0.0,0,0,0,100,0.61,0.0,100,100
2025-03-17,562.77,569.71,562.35,567.15,303269.0,566.385,160588.0,11.291429,NaN,NaN,...,0.0,0.0,0,0,0,100,0.33,0.0,100,100
2025-03-18,564.80,565.02,559.06,561.02,295943.0,561.022,165347.0,11.165000,NaN,NaN,...,0.0,0.0,0,0,0,100,1.42,0.0,100,100
2025-03-19,562.83,570.95,561.63,567.13,355690.0,566.206,201493.0,10.875714,NaN,NaN,...,0.0,0.0,0,0,0,100,0.35,0.0,100,100
2025-03-20,563.33,570.57,562.60,565.49,332328.0,566.528,174877.0,10.567857,NaN,NaN,...,0.0,0.0,0,0,0,100,1.14,0.0,100,100
2025-03-21,559.29,564.89,558.03,563.98,359481.0,561.772,192769.0,9.812143,NaN,NaN,...,0.0,0.0,0,0,0,0,0.11,0.0,0,0
2025-03-24,570.80,575.15,570.20,574.08,329058.0,573.290,165355.0,9.227143,NaN,NaN,...,66.666667,0.0,0,100,0,100,26.84,0.0,100,100


In [11]:
spy.get_frame('1 day').plot()
spy.get_frame('1 hour').plot()
sx.get_frame('1 day').plot()
sx.get_frame('1 hour').plot()

# ----------------------

In [8]:
sx.get_frame('1 day').taPresets.get_ta_list()


scr = 20

scr *=-1
scr

-20

In [9]:
sx.get_frame_data('1 day', colsContains='Score')

""
date
2024-06-21
2024-06-24
2024-06-25
2024-06-26
2024-06-27
...
2025-04-02
2025-04-03
2025-04-04


In [10]:
spy.RUN_DAILY(isMarket=True)

sx.set_ls('SHORT')
sx.RUN_DAILY(spy)

------------ StockX::RUN_DAILY: SPY LONG------------------------------------------------------------------------


ValueError: Invalid ls value: 

In [ ]:
sx.get_frame_data('1 hour', colsContains='Score')

In [ ]:
sx.set_daily_stats(printStats=True)

In [ ]:
sx.get_frame_data('1 day').tail(10)

In [ ]:
sx.get_frame('1 day').plot()

In [ ]:
sx.reqLiveBars() # waits 3 seconds for the data to come in

In [ ]:

data = sx.updateLiveOHLCV(['1 hour'], spy)
display(data)
sx.get_frame('1 hour').plot()

In [ ]:


# ! next do the stock X day data for 5 mins and 2  mins 

In [ ]:
sx.get_live_price()

## 2. Fundamentals

In [ ]:
sx.RUN_FUNDAMENTALS(allowedETFs=['XLY', 'XLK', 'XLC'], maxDaysOld=10)

## 3. Setup

In [ ]:
sx.RUN_SETUP(spy, dataType='ohlcv', forceDownload=False, endDate='2025-02-20')

## 4. PreMarket

In [ ]:
sx.RUN_PRE_MARKET('LONG', timeToRun='09:00', minPreMarketScore=50, displayCharts=True, printStats=True) # runs daily and premarket volume analysis

In [ ]:
sx.display_columns('1 day')
sx.display_columns('1 day', contains='SPY')

## 5. Intraday

In [ ]:
sx.RUN_INTRADAY(updateEverySeconds=60, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=True, logTrades=True)

In [ ]:
import pandas as pd
import strategies.preset_strats as ps
import strategies.signals as sig
import strategies.ta as ta
from typing import Dict, Any, List
from chart.chart import ChartArgs

sx.import_all_market_data(spy)
sx.set_ls('LONG')
sx.setup_TA_1D(lookBack=25, atrSpan=14, sigRow=3, validationRow=4)

if sx.day_score_passed(minScore=50):
    sx.setup_TA_intraday(lookBack=200, atrSpan=50, sigRow=3, validationRow=4)
    
    if sx.pre_market_passed(minScore=50):
        sx.RUN(updateEvery=30, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=False, logTrades=True)




spy.frames['1 day'].plot()
sx.frames['1 day'].plot()
sx.frames['1 hour'].plot()
sx.frames['5 mins'].plot()

In [ ]:
sx.display_columns('5 mins', contains='PB')

In [ ]:
dfx = sx.display_columns('5 mins', contains='PB')
dfx.loc['2025-02-20 09:35:00':].head(50)

 # Stratgey

In [ ]:
ib.disconnect()

In [ ]:
sx.get_frame('1 hour').taPresets.get_ta_list()
# prests.s_1H.passed
#  max rows
import pandas as pd
pd.set_option('display.max_rows', 1000)
sx.get_frame_data('1 hour', colsContains='Score').tail(100)

,Score_s_1H_TODC,Score_s_1H_TODC_passed,Score_sv_1H_TODC,Score_sv_1H_TODC_passed,Score_s_1H_TODC_F,Score_s_1H_TODC_F_passed
date,,,,,,
2025-03-27 18:00:00,11.5,0.0,0,0,11.5,1.0
2025-03-27 19:00:00,10.7,0.0,0,0,10.7,1.0
2025-03-27 20:00:00,10.5,0.0,0,0,10.5,1.0
2025-03-27 21:00:00,10.4,0.0,0,0,10.4,1.0
2025-03-27 22:00:00,10.5,0.0,0,0,10.5,1.0
2025-03-27 23:00:00,10.5,0.0,0,0,10.5,1.0
2025-03-28 00:00:00,48.8,0.0,0,0,48.8,1.0
2025-03-28 08:00:00,50.1,1.0,100,100,50.1,0.0
2025-03-28 09:00:00,50.3,1.0,100,100,50.3,0.0


In [ ]:

import pandas as pd
from dataclasses import dataclass, field
from strategies.signals import Score, MultiSignals, Strategy
from strategies.preset_strats import TAPresets1D, TAPresets1H, ChartArgs
from typing import Dict, Any, List

@dataclass
class Strategy(MultiSignals):
    name: str
    chartArgs: List[ChartArgs] = field(default_factory=list)


    def __post_init__(self):
        self.names = [self.name, f"{self.name}_pctComplete", f"{self.name}_passed", f"{self.name}_meanScore"]
        self.steps = []
        self.rows = [] #  list of dicts for each row
        self.pctComplete = 0
        self.meanScoreOfSteps = 0
        self.current_step = 1
        self.passed = False
        self.scoreList = []
        self.chartArgs = []
        self.count_steps_passed = 0

    
    def add_chart_args(self, meanScoreArgs:ChartArgs, pctCompleteArgs:ChartArgs, scoreArgs:ChartArgs, failArgs:ChartArgs, scoreSubItemArgs:ChartArgs):
        meanScoreArgs.name = f"{self.name}_meanScore"
        meanScoreArgs.columns = [meanScoreArgs.name]
        pctCompleteArgs.name = f"{self.name}_pctComplete"
        pctCompleteArgs.columns = [pctCompleteArgs.name]
        scoreArgs.name = f"{self.name}_score"
        scoreArgs.columns = []
        failArgs.name = f"{self.name}_fail"
        failArgs.columns = []
        scoreSubItemArgs.name = f"{self.name}_subItem"
        scoreSubItemArgs.columns = []
        self.chartArgs = [meanScoreArgs, pctCompleteArgs, scoreArgs, failArgs, scoreSubItemArgs]
        return self
    
    def get_name(self, step:int, objName):
        name = f"{self.name}_{step}_{objName}"
        if name not in self.names:  # Avoid duplicates
            self.names.append(name)
        return name

    def add_step(self, scoreObj:Score, failObj:Score, ifFailStartFromStep:int):
        self.steps.append({'scoreObj': scoreObj, 'failObj': failObj, 'ifFailStartFromStep': ifFailStartFromStep})
        step = len(self.steps)
        self.chartArgs[2].columns += [self.get_name(step, scoreObj.name)]
        self.chartArgs[3].columns += [self.get_name(step, failObj.name)]
        self.chartArgs[4].columns += [self.get_name(step, sig.name) for sig in scoreObj.sigs]
        return self
        
    
    def add_row_item(self, df, obj):
        name = f"{self.name}_{self.current_step}_{obj.name}"
        self.rows[-1][name] = df[obj.name].iat[-1] 
        return self
    
    def add_row_metrics(self):
        self.rows[-1][f"{self.name}_passed"] = self.passed
        self.rows[-1][f"{self.name}_pctComplete"] = self.pctComplete
        self.rows[-1][f"{self.name}_meanScore"] = self.meanScoreOfSteps
        return self
    
    def update_mean_score(self, df, scoreObj):
        score = df[scoreObj.name].iloc[-1]
        self.scoreList += [score]
        self.meanScoreOfSteps = sum(self.scoreList) / len(self.scoreList)

    def update_pct_complete(self):
        self.pctComplete = 0 if self.count_steps_passed == 0 else (self.count_steps_passed / len(self.steps)) * 100

    def reset(self, startFromStep:int=1):
        self.current_step = startFromStep
        self.count_steps_passed = startFromStep - 1
        self.scoreList = self.scoreList[:startFromStep-1] # keep the scores from the failed step
        self.meanScoreOfSteps = 0 if len(self.scoreList) == 0 else sum(self.scoreList) / len(self.scoreList)
        self.update_pct_complete()
        self.passed = False
  


    def _compute_row(self, df: pd.DataFrame) -> pd.Series:
        if self.passed > 0:
            self.reset(1) # reset the strategy if it has passed

        self.rows.append({}) # add a new row to the list of rows

        for i, step in enumerate(self.steps):

            if i + 1 >= self.current_step:
                scoreObj = step['scoreObj']
                failObj = step['failObj']
                scorePassed  = df[scoreObj.name_passed].iloc[-1]
                failed = df[failObj.name_passed].iloc[-1]

                if pd.isna(scorePassed):
                    break

                if failed > 0:
                    self.reset(step['ifFailStartFromStep'])
                    break

                if scorePassed > 0:
                    self.count_steps_passed += 1
                    self.update_mean_score(df, scoreObj)
                    self.add_row_item(df, scoreObj)
                    self.add_row_item(df, failObj)
                    
                    for sig in scoreObj.sigs:
                        self.add_row_item(df, sig)

                    if self.current_step == len(self.steps):
                        self.passed = True
                    else:
                        self.current_step += 1

        self.update_pct_complete()
        self.add_row_metrics()
      
        # Create a Series with NaN values for all expected columns
        result = pd.Series(index=self.names, dtype='float64')
        
        # Fill in the values that we have
        for key, value in self.rows[-1].items():
            if key in result.index:
                result[key] = value
        
        return result

    def compute_signals(self, df: pd.DataFrame) -> pd.DataFrame:
        results = pd.DataFrame(index=df.index, columns=self.names)
        for i in range(len(df)):
            results.iloc[i] = self._compute_row(df.iloc[:i+1])
        return results

#! TODO -- plot so each step can be viewed one ata a time.  Will maybe need to mask the chart df for this sepcial case 
ca = ChartArgs()

prests = sx.get_frame('1 hour').taPresets
strat = Strategy(name='SID').add_chart_args(ca.StratMeanScore, ca.StratPctComplete, ca.StratScores, ca.StratFails,  ca.StratSubItems)
strat.add_step(scoreObj=prests.s_1H, failObj=prests.s_1H_fail, ifFailStartFromStep=1)
strat.add_step(scoreObj=prests.sv_1H, failObj=prests.s_1H_fail, ifFailStartFromStep=1)

strat.steps
results = strat.compute_signals(sx.get_frame_data('1 hour'))
print(f"Strategy: {strat.name}")
# max rows
pd.set_option('display.max_rows', 1000)
display(results.round(2).tail(100))

Strategy: SID


,SID,SID_pctComplete,SID_passed,SID_meanScore,SID_1_Score_s_1H_TODC,SID_1_Score_s_1H_TODC_F,SID_1_VOL_TODC_100,SID_2_Score_sv_1H_TODC,SID_2_Score_s_1H_TODC_F,SID_2_VAL_L_VOL_TODC_100_>_50
date,,,,,,,,,,
2025-03-27 18:00:00,NaN,0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-27 19:00:00,NaN,0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-27 20:00:00,NaN,0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-27 21:00:00,NaN,0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-27 22:00:00,NaN,0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-27 23:00:00,NaN,0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-28 00:00:00,NaN,0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-28 08:00:00,NaN,100.0,True,75.05,50.1,50.1,50.098798,100.0,50.1,100.0
2025-03-28 09:00:00,NaN,100.0,True,75.15,50.3,50.3,50.311249,100.0,50.3,100.0


In [ ]:
strat.chartArgs
strat.rows

[{'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_passed': False, 'SID_pctComplete': 0, 'SID_meanScore': 0},
 {'SID_pas

In [ ]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()


ib = IB()
port = 7496
port = 4002
ib.connect('127.0.0.1', port, clientId=15)

<IB connected to 127.0.0.1:4002 clientId=15>

In [ ]:

from dataclasses import dataclass, field
from typing import Union, List, Literal, Optional
import pandas as pd
import numpy as np
from strategies.signals import MultiSignals, Signals


@dataclass
class Score(MultiSignals):
    name: str = ''
    ls: str = ''
    sigs: List[Signals] = field(default_factory=list)
    cols : Union[str, List[str]] = field(default_factory=list)   
    scoreType: Literal["sum", "min", "max", "any", "all"] = ''
    operator: Literal[">", "<", ">=", "<=", "=="] = ''
    threshold: Union[float, str] = 0 # Can be a value or column name
    invertScoreIfShort: bool = False

    def __post_init__(self):
        # Ensure the name is unique
        self.name = f"S_{self.name}_{self.scoreType}"
        self.name_passed = f"{self.name}_{self.operator}_{self.threshold}"
        self.names = [self.name, self.name_passed]
        self._filtered_cols = None
        
        # Convert columns to list if it's a string
        if isinstance(self.cols, str):
            self.columns = [self.cols]
        
        # Ensure threshold is a string if it's not a float
        if not isinstance(self.threshold, (float, int)):
            self.threshold = str(self.threshold)
        
        # Initialize the score value
        self.val = 0.0

    def _get_filtered_columns(self, df: pd.DataFrame) -> List[str]:
        """Get and cache filtered columns to avoid recomputation."""
        if self._filtered_cols is None:
            if self.cols and self.sigs:
                raise ValueError("Score::Cannot provide both sigs and cols")
            
            # Use signal names if provided, otherwise use column names
            if self.sigs:
                self._filtered_cols = [sig.name for sig in self.sigs]
            else:
                self._filtered_cols = self.cols if self.cols else list(df.columns)
            
        return self._filtered_cols

    def _compute_row(self, df: pd.DataFrame) -> pd.Series:
        """
        Compute the score based on the last row of the dataframe.
        
        Args:
            df: The pandas DataFrame to compute the score from
                (already pre-sliced to include the relevant rows)
        
        Returns:
            A pandas Series with the computed value and pass/fail status
        """
        # Always use the last row of the dataframe
        last_row = df.iloc[-1]
        
        # Get the values from the column(s)
        values = last_row[self._get_filtered_columns(df)]

        # Apply the score type
        if self.scoreType == "sum":
            computed_value = np.sum(values)
        elif self.scoreType == "mean":
            computed_value = np.mean(values)
        elif self.scoreType == "min":
            computed_value = np.min(values)
        elif self.scoreType == "max":
            computed_value = np.max(values)
        elif self.scoreType == "any":
            computed_value = np.any(values)
        elif self.scoreType == "all":
            computed_value = np.all(values)

        # apply score normalization
        if self.normRange:
            computed_value = self.get_score(computed_value)

        if self.invertScoreIfShort and self.ls == 'SHORT':
            computed_value *= -1
        
        # Get the threshold value
        if isinstance(self.threshold, str):
            threshold_value = last_row[self.threshold]
        else:
            threshold_value = self.threshold
        
        # Apply the comparison operator
        if self.operator == ">":
            passed = computed_value > threshold_value
        elif self.operator == "<":
            passed = computed_value < threshold_value
        elif self.operator == ">=":
            passed = computed_value >= threshold_value
        elif self.operator == "<=":
            passed = computed_value <= threshold_value
        elif self.operator == "==":
            passed = computed_value == threshold_value

            
        
        # Create a Series with the computed value and pass status
        # The second column includes the operator and threshold in the name
        return pd.Series({ self.name: round(computed_value,2), self.name_passed: passed})
        

    def compute_signals(self, df: pd.DataFrame) -> pd.DataFrame:
        results = pd.DataFrame(index=df.index, columns=self.names)
        for i in range(len(df)):
            results.iloc[i] = self._compute_row(df.iloc[:i+1])
        return results

import stock
from strategies.preset_strats import  TAPresets1H, TAPresetsBase, ChartArgs
import strategies.ta as ta
import strategies.signals as sig


sx  = stock.StockX(ib, 'UVIX') 
spy = stock.StockX(ib, 'SPY')


@dataclass
class TAPresets1D(TAPresetsBase):
    name: str = '1D'
    ls: str = 'LONG'    
    lookBack: int = 10
    GappedWRBs_normRange:      tuple[int, int] = (0,100)
    GappedPivots_normRange:    tuple[int, int] = (0, 3)
    GappedPastPivot_normRange: tuple[int, int] = (0,100)
    BarSW_normRange:           tuple[int, int] = (-3,3)
    RoomToMove_normRange:      tuple[int, int] = (0,5)
    isSpy: bool = False

    def __post_init__(self):
        super().__post_init__()
        print(f'{self.name} :: {self.ls} :: {self.lookBack}')
        self.MA50   = ta.MA(maCol='close', period=50).add_chart_args(self.ca.MA50)
        self.MA150  = ta.MA(maCol='close', period=150).add_chart_args(self.ca.MA150)
        self.MA200  = ta.MA(maCol='close', period=200).add_chart_args(self.ca.MA200)
        self.l_ma = [self.MA50, self.MA150, self.MA200]
        self.add_to_ta_list(self.l_ma)

        # Signals (Primary)
        self.BarSW = sig.BarSW(ls=self.ls, normRange=self.BarSW_normRange, atrCol=self.ATR.name, lookBack=self.lookBack).add_chart_args(self.ca.BarSW) 
        self.add_to_ta_list([self.BarSW])


def RUN_DAILY(self, ls:str='', spy:object=None, isMarket:bool=False, displayCharts:bool=False, printStats:bool=False, forceDownload:bool=False):
    print(f"------------ StockX::RUN_DAILY: {self.symbol} {self.ls}------------------------------------------------------------------------")

    self.set_ls(ls)
    if isMarket:
        self.setup_frame('1 day', 'ohlcv', duration="200 D", endDateTime='now', isDayFrame=True,  taPresets=TAPresets1D(ls=self.ls, isSpy=True, lookBack=150))
        # self.setup_frame('1 hour', 'ohlcv', duration="15 D", endDateTime='now', isIntradayFrame=True, taPresets=TAPresets1H(ls=self.ls, isSpy=True, lookBack=150))
        self.frames['1 day'].run_ta()
        # self.frames['1 hour'].run_ta()

    else:
        self.setup_frame('1 day', 'ohlcv', duration="200 D", endDateTime='now', isDayFrame=True,      taPresets=TAPresets1D(ls=self.ls, lookBack=100))
        # self.setup_frame('1 hour', 'ohlcv', duration="15 D", endDateTime='now', isIntradayFrame=True, taPresets=TAPresets1H(ls=self.ls, lookBack=100, volChgPctThreshold=50))
        self.import_all_market_data(spy)
        # self.frames['1 day'].run_ta()
        # self.frames['1 hour'].run_ta()

ls= 'LONG'
RUN_DAILY(spy, ls=ls, isMarket=True)
# RUN_DAILY(sx, ls=ls, spy=spy, isMarket=False)

ps = spy.get_frame('1 day').taPresets
df = spy.get_frame_data('1 day')
print(df.columns)

sc = Score(name='TEST', ls='LONG', sigs=[ps.MA50, ps.MA150], cols=None, scoreType='sum', operator='>=', threshold=58.8, normRange=None, lookBack=10).add_chart_args(ps.ca.MA50)   
sc2 = Score(name='TEST', ls='LONG', sigs=[ps.MA50, ps.MA150], cols=None, scoreType='min', operator='>', threshold=58.8, normRange=(0,1000), lookBack=10)   
display(sc.compute_signals(df).tail(20), df.tail(20))
display(sc2.compute_signals(df).tail(20))

------------ StockX::RUN_DAILY: SPY ------------------------------------------------------------------------
1D :: LONG :: 150
SPY 1 day 200 D :: 2024-06-20 00:00:00 to 2025-04-07 00:00:00 rows: 200, missing: 0, Data stored as 1 day and resampled to 1 day
Index(['open', 'high', 'low', 'close', 'volume', 'average', 'barCount',
       'ATR_14', 'HP_hi_10', 'LP_lo_10', 'HP_hi_3', 'LP_lo_3', 'Res_1',
       'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower',
       'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper',
       'Sup_2_Lower', 'MA_cl_50', 'MA_cl_150', 'MA_cl_200', 'BarSW'],
      dtype='object')


,S_TEST_sum,S_TEST_sum_>=_58.8
date,,
2025-03-11,1176.4,True
2025-03-12,1175.82,True
2025-03-13,1175.15,True
2025-03-14,1174.93,True
2025-03-17,1174.8,True
2025-03-18,1174.51,True
2025-03-19,1174.24,True
2025-03-20,1173.8,True
2025-03-21,1173.44,True


,open,high,low,close,volume,average,barCount,ATR_14,HP_hi_10,LP_lo_10,...,Sup_1,Sup_1_Upper,Sup_1_Lower,Sup_2,Sup_2_Upper,Sup_2_Lower,MA_cl_50,MA_cl_150,MA_cl_200,BarSW
date,,,,,,,,,,,,,,,,,,,,,
2025-03-11,559.40,564.02,552.02,555.92,553801,557.709,303647,10.993571,NaN,NaN,...,542.44,542.44,539.44,510.27,510.27,510.27,594.7830,581.612467,NaN,-11.643168
2025-03-12,562.17,563.11,553.69,558.87,418632,559.312,229036,11.332857,NaN,NaN,...,542.44,542.44,539.44,510.27,510.27,510.27,593.9336,581.889067,NaN,-3.470734
2025-03-13,558.49,559.11,549.68,551.42,400069,553.292,222477,11.232857,NaN,549.68,...,549.68,549.68,539.44,510.27,510.27,510.27,593.0618,582.084200,NaN,-19.318326
2025-03-14,556.11,563.83,555.50,562.81,364382,560.546,191800,11.360714,NaN,NaN,...,549.68,549.68,539.44,510.27,510.27,510.27,592.5536,582.378533,NaN,19.0569
2025-03-17,562.77,569.71,562.35,567.15,303269,566.385,160588,11.291429,NaN,NaN,...,565.16,565.16,564.20,542.44,542.44,539.44,592.1750,582.621867,NaN,9.771424
2025-03-18,564.80,565.02,559.06,561.02,295943,561.022,165347,11.165000,NaN,NaN,...,549.68,549.68,539.44,510.27,510.27,510.27,591.7026,582.808733,NaN,-8.687864
2025-03-19,562.83,570.95,561.63,567.13,355690,566.206,201493,10.875714,NaN,NaN,...,565.16,565.16,564.20,542.44,542.44,539.44,591.2062,583.034467,NaN,9.164149
2025-03-20,563.33,570.57,562.60,565.49,332328,566.528,174877,10.567857,NaN,NaN,...,565.16,565.16,564.20,542.44,542.44,539.44,590.6088,583.190800,NaN,-0.047313
2025-03-21,559.29,564.89,558.03,563.98,359481,561.772,192769,9.812143,NaN,NaN,...,549.68,549.68,539.44,510.27,510.27,510.27,590.1158,583.325667,NaN,16.527141


,S_TEST_min,S_TEST_min_>_58.8
date,,
2025-03-11,58.16,False
2025-03-12,58.19,False
2025-03-13,58.21,False
2025-03-14,58.24,False
2025-03-17,58.26,False
2025-03-18,58.28,False
2025-03-19,58.3,False
2025-03-20,58.32,False
2025-03-21,58.33,False


In [ ]:
values= [20,30]

np.min(values)

20